### Plot aligned variables against calibrated data 
To run this Notebook:
- Install with instructions at https://bitbucket.org/mbari/auv-python
- `cd auv-python`
- `poetry install && poetry shell`
- `cd notebooks && jupyter notebook 2.0-mpm-align.ipynb`

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pylab as plt
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store']);
display(auv_picker);

In [ ]:
# Pick auv mission on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [ ]:
mission = mission_picker.value

netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
    
cal_nc = f"{auv_name}_{mission}_cal.nc"
align_nc = f"{auv_name}_{mission}_align.nc"
cal_ds = xr.open_dataset(os.path.join(netcdfs_dir, cal_nc))
align_ds = xr.open_dataset(os.path.join(netcdfs_dir, align_nc))
display(cal_ds)
display(align_ds)

In [ ]:
%matplotlib widget

plt.figure(figsize=(9,2))
plt.plot(cal_ds['ctd_time'],
         cal_ds['ctd_temperature'], '.',
         align_ds['ctd_time'],
         align_ds['ctd_temperature'])
plt.legend(('Calibrated', 'Aligned'))
title = "Validating alignment of CTD temperature"
plt.title(title)
plt.grid()

In [ ]:
%matplotlib widget

plt.figure(figsize=(9,2))
plt.plot(cal_ds['navigation_time'],
         cal_ds['navigation_latitude'], '.',
         align_ds['ctd_time'],
         align_ds['ctd_latitude'],)
plt.legend(('Calibrated', 'Aligned'))
title = "Validating nudging of latitude"
plt.title(title)
plt.grid()

In [ ]:
%matplotlib widget

plt.figure(figsize=(9,2))
plt.plot(cal_ds['ctd_time'],
         cal_ds['ctd_oxygen_mll'], '.',
         align_ds['ctd_time'],
         align_ds['ctd_oxygen_mll'])
plt.legend(('Calibrated', 'Aligned'))
title = "Validating CTD oxygen_mll"
plt.title(title)
plt.grid()

The align process matches instrument date to the time values of the coordinate data (depth, latitude, and longitude).  Sometimes the instrument data time range exceeds the time range of the coordinate data. We have used `interp1d()`'s `fill_value` property to extrapolate the coordinate values to the temporal edges of the instrument data, but this may be wrong. This became qute apparent with dorado 2020.181.00 (essentially a failed mission that is not processed by legacy Matlab, but is in the missionlogs archive). Let's plot some of the instrument data with the coordinate data to explore the problem of using `"extrapolate"` for `fill_value`.

In [ ]:
%matplotlib widget

plt.figure(figsize=(9,2))
plt.plot(cal_ds['depth_time'],
         cal_ds['depth_filtdepth'], '.',
         align_ds['navigation_time'],
         align_ds['navigation_depth'],
         align_ds['hs2_time'],
         align_ds['hs2_depth'], '--',
         align_ds['ctd_time'],
         align_ds['ctd_depth'], '.-',
         align_ds['ctd_time'],
         align_ds['ctd_salinity'], '*-',
         #align_ds['ctd_time'],
         #align_ds['ctd_salinity'], '--'
         )
plt.legend(('Original depth_filtdepth', 'Aligned navigation', 'Aligned hs2', 'Aligned ctd'))
title = "Validating Interpolation & Extrapolation of depth"
plt.title(title)
plt.grid()